In [51]:
import pandas as pd
import pgeocode
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import Word
import datetime

Đọc dữ liệu

Điều chỉnh lại tên các cột

In [52]:
df = pd.read_json('./jobs_data/data.json')
df = df.rename(columns={'job_type:':'duration'})
df = df.rename(columns=lambda x: x.replace(':', ''))
df.iloc[15:20]


,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information
15,"Agriculture, Fishing & Forestry Jobs",Minibus Driver - Seasonal,17 May 2023,Full time,16 June 2023,"Ross-On-Wye, Herefordshire, HR9 7TF",Homme Soft Fruit Ltd,Temporary,HSF,£10.78 per hour,Overtime after 60 hours
16,"Agriculture, Fishing & Forestry Jobs",Senior Landscaping Quantity Surveyor,17 May 2023,Full time,16 June 2023,"Chelmsford, Essex, CM2 5TT",Professional Personnel Management Ltd,Permanent,SURVME/CM_1684334311,"£40,000 to £50,000 per year","vehicle, fuel card, package"
17,"Agriculture, Fishing & Forestry Jobs",Head of Conservation Services,17 May 2023,Full time,12 June 2023,"Battersea, South West London",Bat Conservation Trust,Permanent,HoCS,"£37,500 to £39,000 per year, pro rata",NaN
18,Property Jobs,Asset Portfolio Officer,19 May 2023,Full time,12 June 2023,"Halifax, West Yorkshire",Calderdale Council,Permanent,CC466,"£37,261 to £40,478 per year",NaN
19,Property Jobs,Contracts Manager,19 May 2023,Full time,26 May 2023,"Loughborough, Leicestershire, LE11 5AA",Travail Employment Group,Permanent,46004237_1684506584,"£21,000 per year","Casual dress, cycle to work scheme"


In [53]:
df.describe()

,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information
count,77771,77771,77771,77771,77771,77771,77771,77771,77771,66688,35044
unique,25,37606,31,2,60,33587,4207,4,65846,10442,11855
top,Other/General Jobs,Support Worker,12 May 2023,Full time,26 May 2023,"London, UK",NHS Jobs,Permanent,,Not specified,Competitive
freq,14735,1071,5948,58799,4393,308,7464,56904,9651,24013,5626


In [54]:
print("Duplications: ", len(df) - len(df.drop_duplicates()))
print("NA: ", len(df) - len(df.dropna(how='all')))
df = df.drop_duplicates()
df.describe()

Duplications:  789
NA:  0


,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information
count,76982,76982,76982,76982,76982,76982,76982,76982,76982,66270,34886
unique,25,37606,31,2,60,33587,4207,4,65846,10442,11855
top,Other/General Jobs,Support Worker,19 May 2023,Full time,26 May 2023,"London, UK",NHS Jobs,Permanent,,Not specified,Competitive
freq,14335,1058,5913,58538,4331,304,7464,56562,9046,23902,5621


In [55]:
data_null = df.isnull().sum()
data_null

job_type                             0
job_name                             0
posting_date                         0
hours                                0
closing_date                         0
location                             0
company                              0
duration                             0
job_reference                        0
salary                           10712
additional_salary_information    42096
dtype: int64

<h2> Cleaning  job_type</h2>

In [56]:
print("""Số lượng row mà job_type chứa từ "Job": """,df['job_type'].str.contains('Jobs', case=False).sum())
df['job_type'] = df['job_type'].str.replace(" Jobs", '') 

Số lượng row mà job_type chứa từ "Job":  76982


<h1>Cleaning date</h1>

<h3> Chuyển sang timestamp </h3>

In [57]:
df['posting_date'] = pd.to_datetime(df['posting_date'])
df['closing_date'] = pd.to_datetime(df['closing_date'])
df.iloc[20:25]

,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information
20,Property,Onsite Recruitment Coordinator,2023-05-19,Full time,2023-06-18,IV30 1BD,Staffline,Permanent,ACIEL-36959,NaN,NaN
21,Property,ALS Coordinator,2023-05-19,Full time,2023-06-18,"Waltham Forest, London, E174JB",Morgan Hunt UK Ltd,Temporary,TR/022078_1684504137,£14 to £16 per hour,NaN
22,Property,Property Surveyor,2023-05-19,Full time,2023-06-04,"Cannock, Staffordshire",Cannock Chase Council,Permanent,CH117,"£28,371 to £32,020 per year",NaN
23,Scientific & QA,Research Fellow in Emerging Topics in Robotics,2023-05-22,Full time,2023-06-14,"Hatfield, Hertfordshire",University of Hertfordshire,Contract,051776,NaN,NaN
24,Scientific & QA,7501 - Postdoctoral Research Fellow,2023-05-22,Full time,2023-06-12,"Edinburgh, Scotland",University of Edinburgh,Temporary,7501,"£36,333 to £43,155 per year",NaN


In [58]:
print("Date outlier count: ", len(df[df['posting_date'] > df['closing_date']]))
print("Posting outlier count: ", len(df[df['posting_date'] > datetime.datetime.now()]))

Date outlier count:  0
Posting outlier count:  0


<h1> Cleaning location </h1>

In [60]:
nomi = pgeocode.Nominatim('gb')
def convertPostCode(code):
    if(type(nomi.query_postal_code(code)['place_name']) == str  ) : return  nomi.query_postal_code(code)['place_name']
    else: return code
df['location'] = df['location'].apply(convertPostCode)
df.iloc[20:25]

,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information
20,Property,Onsite Recruitment Coordinator,2023-05-19,Full time,2023-06-18,"Burghead, Glenlatterach, Bishopmill, Whitewrea...",Staffline,Permanent,ACIEL-36959,NaN,NaN
21,Property,ALS Coordinator,2023-05-19,Full time,2023-06-18,"Waltham Forest, London, E174JB",Morgan Hunt UK Ltd,Temporary,TR/022078_1684504137,£14 to £16 per hour,NaN
22,Property,Property Surveyor,2023-05-19,Full time,2023-06-04,"Cannock, Staffordshire",Cannock Chase Council,Permanent,CH117,"£28,371 to £32,020 per year",NaN
23,Scientific & QA,Research Fellow in Emerging Topics in Robotics,2023-05-22,Full time,2023-06-14,"Hatfield, Hertfordshire",University of Hertfordshire,Contract,051776,NaN,NaN
24,Scientific & QA,7501 - Postdoctoral Research Fellow,2023-05-22,Full time,2023-06-12,"Edinburgh, Scotland",University of Edinburgh,Temporary,7501,"£36,333 to £43,155 per year",NaN


<h1> Cleaning salary</h1>

In [69]:
df['salary'] =  df['salary'].fillna('Not specified')
print("Not specified count: ", len(df[df['salary'] == 'Not specified']))
print("Specified count: ", len(df[df['salary'] != 'Not specified']))
print("""Contain "per" keyword count: """, len(df[df['salary'].apply(lambda x: "per" in x)]))
print("""Contain "to" keyword count: """, len(df[df['salary'].apply(lambda x: "to" in x)]))
print("""Contain "£" keyword count: """, len(df[df['salary'].apply(lambda x: "to" in x)]))

Not specified count:  34614
Specified count:  42368
Contain "per" keyword count:  42368
Contain "to" keyword count:  25427
Contain "£" keyword count:  25427


<p> Như vậy ở các hàng có salary khác "Not specified" thì đều có "per".</p>
<p> Đơn vị tiền tệ đều là £ </p>

In [73]:
def salary_prc(salary):
    if(salary == 'Not specified'): return []
    words = salary.split()
    keyword_index1 = words.index('to') if('to' in words) else len(words)
    keyword_index2 = words.index('per')
    per_unit = [words[keyword_index2 + 1]]
    if keyword_index1 < len(words) - 1:
        salary_range = [words[keyword_index1-1], words[keyword_index1+1]]
    else:
        salary_range= [words[keyword_index2-1], words[keyword_index2-1]]
    return salary_range + per_unit
    

df['salary_process'] = df['salary'].apply(salary_prc)
df[['salary_from','salary_to', 'per_unit']] = df['salary_process'].apply(lambda x: pd.Series(x))
df.iloc[0:10]

C:\Users\Admin\AppData\Local\Temp\ipykernel_15112\2648121286.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['salary_from','salary_to', 'per_unit']] = df['salary_process'].apply(lambda x: pd.Series(x))


,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information,salary_process,salary_from,salary_to,per_unit
0,"Agriculture, Fishing & Forestry",Harvest Temp - Site Operative,2023-05-19,Full time,2023-06-16,"Cranswick, Driffield",Frontier Agriculture Ltd,Temporary,1788,Not specified,NaN,[],NaN,NaN,NaN
1,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Wiltshire, SP1 1TT",Momentum Instore Installations,Contract,ORG2411-HW797397SalFMSE,Not specified,NaN,[],NaN,NaN,NaN
2,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Isle of Wight, PO30 2NB",Momentum Instore Installations,Contract,ORG2411-HW797420NewFMSE,Not specified,NaN,[],NaN,NaN,NaN
3,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Wiltshire, SP1 1TT",Momentum Instore Installations,Contract,ORG2411-HW797406SalFMSE,Not specified,NaN,[],NaN,NaN,NaN
4,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Perth and Kinross, PH1 2NR",Momentum Instore Installations,Contract,ORG2411-HW797403PerFMSE,Not specified,NaN,[],NaN,NaN,NaN
5,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Perth and Kinross, DD2 1UR",Momentum Instore Installations,Contract,ORG2411-HW797399DunFMSE,Not specified,NaN,[],NaN,NaN,NaN
6,Property,Inventory Clerk,2023-05-20,Part time,2023-06-19,"Bedminster, Bristol",Start 2 Finish International Ltd,Permanent,,Not specified,Commission Based,[],NaN,NaN,NaN
7,Scientific & QA,Lecturer in Marine Ecosystem Services (Teachin...,2023-05-22,Full time,2023-06-16,"Pont Pen-y-benglog, Bethesda, Port Penrhyn, Do...",Bangor University,Permanent,BU03243,"£44,414 to £52,841 per year",NaN,"[£44,414, £52,841, year]","£44,414","£52,841",year
8,Scientific & QA,Chemistry Analyst,2023-05-22,Full time,2023-06-21,X35,crooton,Permanent,CTNJM3045D-40750,Not specified,NaN,[],NaN,NaN,NaN
9,Scientific & QA,Chemistry Analyst,2023-05-22,Full time,2023-06-21,E91,crooton,Permanent,CTNJM3045CL-40750,Not specified,NaN,[],NaN,NaN,NaN


In [77]:
df['per_unit'].unique()

array([nan, 'year', 'hour', 'year,', 'week', 'day', 'month', 'fortnight'],
      dtype=object)

In [ ]:
def handle_salary(salary,per_unit):
    
    if(type(salary) == str):
        return float(salary.replace('£', '').replace(',','')) 
    else: return salary

In [84]:
df.pop('salary_process')
df.pop('salary')
df['salary_to_1'] = df.apply(lambda x: x.replace('£', '') if type(x) == str else x)
df['salary_to_1']
df.iloc[0:10]

,job_type,job_name,posting_date,hours,closing_date,location,company,duration,job_reference,salary,additional_salary_information,salary_from,salary_to,per_unit,salary_to_1
0,"Agriculture, Fishing & Forestry",Harvest Temp - Site Operative,2023-05-19,Full time,2023-06-16,"Cranswick, Driffield",Frontier Agriculture Ltd,Temporary,1788,Not specified,NaN,NaN,NaN,NaN,NaN
1,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Wiltshire, SP1 1TT",Momentum Instore Installations,Contract,ORG2411-HW797397SalFMSE,Not specified,NaN,NaN,NaN,NaN,NaN
2,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Isle of Wight, PO30 2NB",Momentum Instore Installations,Contract,ORG2411-HW797420NewFMSE,Not specified,NaN,NaN,NaN,NaN,NaN
3,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Wiltshire, SP1 1TT",Momentum Instore Installations,Contract,ORG2411-HW797406SalFMSE,Not specified,NaN,NaN,NaN,NaN,NaN
4,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Perth and Kinross, PH1 2NR",Momentum Instore Installations,Contract,ORG2411-HW797403PerFMSE,Not specified,NaN,NaN,NaN,NaN,NaN
5,"PR, Advertising & Marketing",FIELD MARKETING – SELF EMPLOYED,2023-05-22,Full time,2023-06-19,"Perth and Kinross, DD2 1UR",Momentum Instore Installations,Contract,ORG2411-HW797399DunFMSE,Not specified,NaN,NaN,NaN,NaN,NaN
6,Property,Inventory Clerk,2023-05-20,Part time,2023-06-19,"Bedminster, Bristol",Start 2 Finish International Ltd,Permanent,,Not specified,Commission Based,NaN,NaN,NaN,NaN
7,Scientific & QA,Lecturer in Marine Ecosystem Services (Teachin...,2023-05-22,Full time,2023-06-16,"Pont Pen-y-benglog, Bethesda, Port Penrhyn, Do...",Bangor University,Permanent,BU03243,"£44,414 to £52,841 per year",NaN,"£44,414","£52,841",year,"52,841"
8,Scientific & QA,Chemistry Analyst,2023-05-22,Full time,2023-06-21,X35,crooton,Permanent,CTNJM3045D-40750,Not specified,NaN,NaN,NaN,NaN,NaN
9,Scientific & QA,Chemistry Analyst,2023-05-22,Full time,2023-06-21,E91,crooton,Permanent,CTNJM3045CL-40750,Not specified,NaN,NaN,NaN,NaN,NaN


In [86]:
float(df['salary_to_1'][7].replace(',',''))

52841.0